In [ ]:
!pip install gradio openai

In [5]:
!pip install openai==0.28

In [16]:
import gradio as gr
import openai

In [17]:
# path to project: Edit here to point to your Drive
drive = '/content/drive/MyDrive/'
project_dir = 'Colab Notebooks/IronHacks2024_Data_Analytics/DA_Week_8'
COLAB_PROJECT_PATH = drive+project_dir+'/LLM_Mini_Project'

In [18]:
import os
#Mounted Google Drive
using_colab = False
if 'google.colab' in str(get_ipython()):
    from google.colab import drive
    # Mount Google Drive
    drive.mount('/content/drive', force_remount=True)
    using_colab = True

if using_colab and os.path.exists(COLAB_PROJECT_PATH):
    print('found colab path; redirecting to main project directory')
    os.chdir(COLAB_PROJECT_PATH)

Mounted at /content/drive
found colab path; redirecting to main project directory


In [23]:
# You can save your key in a text file and read it
import os

with open('gpt_key.txt') as file:
    api_key = file.read().strip('\n')

# load and set our key
openai.api_key = api_key

In [3]:
os.listdir()

['3_Musketeer_presentation.gslides', 'gpt_key.txt', '3_musketeer_LLM.ipynb']

In [69]:
# Lets rewrite the gpt_response function to include possible parameters
def gpt_response(inp, message_history, **params):
    # We save the user's input
    message_history.append({"role": "user", "content": f"{inp}"})

    # Generate a response from the chatbot model
    completion_params = {
        "model": "gpt-3.5-turbo",
        "messages": message_history,
        **params  # Include additional parameters
    }

    completion = openai.ChatCompletion.create(**completion_params) # Include additional parameters

    # We save the assistant response
    message_history.append({"role": "assistant", "content": f"{completion.choices[0].message.content}"})

    return message_history

In [74]:
prompt = 'Generate closing statements from a presidential debate between two fictitious characters running for USA presidential elections'
params = {'temperature': 1.5} # .5, 1, 1.5
MAX_TOKENS = 350 if params['temperature'] >= 1.5 else 250

In [75]:
message_history = [{"role": "system", "content": prompt},
                   {"role": "assistant", "content": f"OK"}]

def gpt_response_ui(inp):
    global message_history

    message_history = gpt_response(inp, message_history, max_tokens = MAX_TOKENS)

    # Get pairs (as tuples) of msg["content"] from message history,representing one exchange between the user and the chatbot,
    # skipping the pre-prompt
    response = [(message_history[i]["content"], message_history[i+1]["content"]) for i in range(2, len(message_history)-1, 2)]
    return response

In [76]:
with gr.Blocks() as demo: #creates a Gradio interface

    chatbot = gr.Chatbot() #creates a chatbot instance

    with gr.Row(): #creates a row within the Gradio interface to contain components
        txt = gr.Textbox(show_label=False, placeholder="Enter text and press enter") #text input field
        # `show_label=False` parameter hides the label associated with the textbox

    txt.submit(gpt_response_ui, txt, chatbot) #sets the submit action to the `gpt_response` function
    txt.submit(lambda :"", None, txt) #this clears the textbox when the user submits their input

In [77]:
demo.launch(share=True) #To create a public link, set `share=True` in `launch()`.

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://0b962d18a898cb7d64.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
